<a href="https://colab.research.google.com/github/1X-HE/OKOKqm2/blob/main/heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

# Read CSV file
print("Reading CSV file...")
df = pd.read_csv('MPS Borough Level Crime (most recent 24 months).csv')

# Get month columns (starting from column 4 are month data)
month_columns = [col for col in df.columns if
                 col.startswith('2023') or col.startswith('2024') or col.startswith('2025')]

# Count occurrences of each MajorText (by row count)
major_text_counts = df['MajorText'].value_counts()
print("\nCrime type occurrence statistics:")
print(major_text_counts)

# Get top 3 and last crime types
top_3 = major_text_counts.head(3).index.tolist()
last_one = major_text_counts.tail(1).index.tolist()
selected_crimes = top_3 + last_one

print(f"\nSelected crime types:")
print(f"Top 3: {top_3}")
print(f"Last 1: {last_one}")
# London borough coordinates (approximate center points)
borough_coords = {
    'Barking and Dagenham': [51.5369, 0.0819],
    'Barnet': [51.6252, -0.2079],
    'Bexley': [51.4569, 0.1508],
    'Brent': [51.5588, -0.2817],
    'Bromley': [51.4050, 0.0149],
    'Camden': [51.5450, -0.1431],
    'City of London': [51.5155, -0.0903],
    'Croydon': [51.3762, -0.0972],
    'Ealing': [51.5133, -0.3081],
    'Enfield': [51.6522, -0.0820],
    'Greenwich': [51.4894, 0.0648],
    'Hackney': [51.5450, -0.0547],
    'Hammersmith and Fulham': [51.4924, -0.2339],
    'Haringey': [51.5900, -0.1100],
    'Harrow': [51.5806, -0.3427],
    'Havering': [51.5777, 0.1839],
    'Hillingdon': [51.5352, -0.4495],
    'Hounslow': [51.4676, -0.3615],
    'Islington': [51.5369, -0.1030],
    'Kensington and Chelsea': [51.4994, -0.1938],
    'Kingston upon Thames': [51.4105, -0.3004],
    'Lambeth': [51.4952, -0.1119],
    'Lewisham': [51.4652, -0.0139],
    'Merton': [51.4098, -0.2108],
    'Newham': [51.5255, 0.0356],
    'Redbridge': [51.5900, 0.0819],
    'Richmond upon Thames': [51.4613, -0.3004],
    'Southwark': [51.5033, -0.0897],
    'Sutton': [51.3600, -0.1917],
    'Tower Hamlets': [51.5200, -0.0293],
    'Waltham Forest': [51.5856, -0.0118],
    'Wandsworth': [51.4571, -0.1920],
    'Westminster': [51.4975, -0.1357],
}
# Create heatmap for each selected crime type
for crime_type in selected_crimes:
    print(f"\nProcessing crime type: {crime_type}")

    # Filter data for this crime type
    crime_df = df[df['MajorText'] == crime_type].copy()

    # Calculate total crimes for each borough (sum of all months)
    borough_totals = {}
    for borough in crime_df['BoroughName'].unique():
        borough_data = crime_df[crime_df['BoroughName'] == borough]
        total = borough_data[month_columns].sum().sum()
        borough_totals[borough] = total

    print(f"Total crimes by borough (top 5):")
    sorted_totals = sorted(borough_totals.items(), key=lambda x: x[1], reverse=True)
    for borough, total in sorted_totals[:5]:
        print(f"  {borough}: {total}")

    # Create base map (centered on London)
    m = folium.Map(
        location=[51.5074, -0.1278],  # London center coordinates
        zoom_start=10,
        tiles='OpenStreetMap'
    )

    # Prepare heatmap data
    heat_data = []
    max_crime = max(borough_totals.values()) if borough_totals else 1
    min_crime = min(borough_totals.values()) if borough_totals else 1
  # Generate multiple points for each borough to enhance heatmap effect
    for borough, total in borough_totals.items():
        if borough in borough_coords:
            lat, lon = borough_coords[borough]
            # Normalized weight (0-1)
            normalized_weight = (total - min_crime) / (max_crime - min_crime) if max_crime > min_crime else 1.0
            # Add points with different density based on crime count
            num_points = max(1, int((total / max_crime) * 50))  # Max 50 points
            for _ in range(num_points):
                # Add random offset for more natural heatmap
                offset_lat = lat + np.random.uniform(-0.01, 0.01)
                offset_lon = lon + np.random.uniform(-0.01, 0.01)
                heat_data.append([offset_lat, offset_lon, normalized_weight])

    # Add heatmap layer
    if heat_data:
        HeatMap(
            heat_data,
            min_opacity=0.3,
            max_zoom=18,
            radius=30,
            blur=20,
            gradient={
                0.0: 'blue',
                0.3: 'cyan',
                0.5: 'lime',
                0.7: 'yellow',
                0.9: 'orange',
                1.0: 'red'
            }
        ).add_to(m)

    # Add markers showing detailed data for each borough, with red for highest crime areas
    for borough, total in borough_totals.items():
        if borough in borough_coords:
            lat, lon = borough_coords[borough]
            # Set color based on crime count: red for highest, others proportionally from blue to green
            if total == max_crime:
                color = 'red'
                radius = 15
            else:
                ratio = (total - min_crime) / (max_crime - min_crime) if max_crime > min_crime else 0
                if ratio > 0.7:
                    color = 'orange'
                    radius = 12
                elif ratio > 0.4:
                    color = 'yellow'
                    radius = 10
                else:
                    color = 'blue'
                    radius = 8

            folium.CircleMarker(
                location=[lat, lon],
                radius=radius,
                popup=f"<b>{borough}</b><br>Total crimes: {total:,}",
                tooltip=f"{borough}: {total:,}",
                color='black',
                weight=2,
                fill=True,
                fillColor=color,
                fillOpacity=0.7
            ).add_to(m)

    # Add title and legend
    title_html = f'''
                <div style="position: fixed;
                             top: 10px; left: 50%; transform: translateX(-50%);
                             width: 600px; height: 120px;
                             background-color: white; border: 2px solid black;
                             z-index:9999; font-size:14px; padding: 10px;
                             border-radius: 5px; box-shadow: 0 0 15px rgba(0,0,0,0.3);">
                 <h3 align="center" style="margin: 5px 0; font-size:18px;"><b>{crime_type}</b></h3>
                 <p align="center" style="margin: 5px 0; font-size:14px;">
                     <b>Borough with highest total crimes:</b> {sorted_totals[0][0]} ({sorted_totals[0][1]:,} cases)
                 </p>
                 <p align="center" style="margin: 5px 0; font-size:12px; color: #666;">
                     Color legend: Blue(few) → Cyan → Green → Yellow → Orange → Red(many)
                 </p>
                 </div>
                 '''
    m.get_root().html.add_child(folium.Element(title_html))

    # Save map
    filename = f"crime_heatmap_{crime_type.replace(' ', '_').replace('/', '_')}.html"
    m.save(filename)
    print(f"Map saved as: {filename}")

print("\nAll heatmaps have been generated!")




Reading CSV file...

Crime type occurrence statistics:
MajorText
VIOLENCE AGAINST THE PERSON             168
THEFT                                   136
VEHICLE OFFENCES                        135
PUBLIC ORDER OFFENCES                   129
BURGLARY                                104
ROBBERY                                  68
SEXUAL OFFENCES                          68
DRUG OFFENCES                            68
ARSON AND CRIMINAL DAMAGE                68
POSSESSION OF WEAPONS                    34
MISCELLANEOUS CRIMES AGAINST SOCIETY     34
FRAUD AND FORGERY                        32
NFIB FRAUD                               12
Name: count, dtype: int64

Selected crime types:
Top 3: ['VIOLENCE AGAINST THE PERSON', 'THEFT', 'VEHICLE OFFENCES']
Last 1: ['NFIB FRAUD']

Processing crime type: VIOLENCE AGAINST THE PERSON
Total crimes by borough (top 5):
  Croydon: 22509
  Westminster: 22238
  Tower Hamlets: 20317
  Newham: 20309
  Ealing: 19734
Map saved as: crime_heatmap_VIOLENCE_AGAINST_